# Batch Place Cell Analysis

Run the full analysis pipeline on multiple datasets, save each as a
`.pcellbundle` for later visualization, and summarize how many units pass
significance and stability tests.

Bundles are named after the data YAML stem (e.g. `WL25_20251201.pcellbundle`).
If a bundle with that name already exists, a `_1`, `_2`, … suffix is appended.

In [ ]:
import sys
from pathlib import Path

import pandas as pd
from tqdm.notebook import tqdm

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from placecell.dataset import PlaceCellDataset, unique_bundle_path
from placecell.visualization import plot_session_summary

## Configuration

Edit `DATA_PATHS` to list your dataset YAML files.
Set `BUNDLE_DIR` to where bundles should be saved.

In [ ]:
CONFIG_ID = "pcell_config"

DATA_ROOT = Path(
    "/Volumes/ProcData/minizero_analysis/202512round/202511_analysis_placecell/"
)

DATA_PATHS = [
    DATA_ROOT / "20251201/WL25/WL25_20251201.yaml",
    DATA_ROOT / "20251202/WL25/WL25_20251202.yaml",
    DATA_ROOT / "20251203/WL25/WL25_20251203.yaml",
    DATA_ROOT / "20251204/WL25/WL25_20251204.yaml",
    DATA_ROOT / "20251205/WL25/WL25_20251205.yaml",
]

BUNDLE_DIR = DATA_ROOT / "bundles"

## Run Analysis & Save Bundles

Processes each dataset and saves a `.pcellbundle` directory.
Uses `stability_method='shuffle'` so we get both the shuffle p-value
and raw correlation from a single analysis run.

In [ ]:
BUNDLE_DIR.mkdir(parents=True, exist_ok=True)

rows = []
for data_path in DATA_PATHS:
    name = data_path.stem
    print(f"\n{'='*60}")
    print(f"Processing: {name}")
    print(f"{'='*60}")

    ds = PlaceCellDataset.from_yaml(CONFIG_ID, data_path)

    # Force shuffle mode so we get both stability metrics
    new_spatial = ds.cfg.behavior.spatial_map.model_copy(
        update={"stability_method": "shuffle"}
    )
    new_behavior = ds.cfg.behavior.model_copy(
        update={"spatial_map": new_spatial}
    )
    ds.cfg = ds.cfg.model_copy(update={"behavior": new_behavior})

    ds.load()
    ds.deconvolve(
        progress_bar=lambda x, **kw: tqdm(x, desc="Deconvolving", **kw),
    )
    ds.match_events()
    ds.compute_occupancy()
    ds.analyze_units(
        progress_bar=lambda x, **kw: tqdm(x, desc="Analyzing", **kw),
    )

    # Save bundle
    bundle_path = unique_bundle_path(BUNDLE_DIR, name)
    ds.save_bundle(bundle_path)
    print(f"  Saved: {bundle_path}")

    row = {"dataset": name, "bundle": str(bundle_path.name), **ds.summary()}
    rows.append(row)

    print(
        f"  Significant:          {row['n_sig']}/{row['n_total']}\n"
        f"  Stable (threshold):   {row['n_stable_thresh']}/{row['n_total']}\n"
        f"  Stable (shuffle):     {row['n_stable_shuffle']}/{row['n_total']}\n"
        f"  Both (threshold):     {row['n_both_thresh']}/{row['n_total']}\n"
        f"  Both (shuffle):       {row['n_both_shuffle']}/{row['n_total']}"
    )

## Summary Table

In [ ]:
df = pd.DataFrame(rows)
display(df[["dataset", "bundle", "n_total", "n_sig",
            "n_stable_thresh", "n_stable_shuffle",
            "n_both_thresh", "n_both_shuffle"]])

## Across-Session Plot

In [ ]:
import matplotlib.pyplot as plt

plot_session_summary(df)
plt.show()